# DNBLab Jupyter Notebook Tutorial

## OAI(2) - Schnittstellenabfragen, Datenauslieferung und Ergebnisanzeige

Diese DNBLab-Tutorials beschreiben Beispielabfragen über die OAI2- und OAI-Schnittstelle mit Python. In der Jupyter Notebook Umgebung kann der dokumentierte Code direkt ausgeführt und angepasst werden. Die Tutorials umfassen die exemplarische Anfrage und Ausgabe der Daten in den verschiedenen Metadatenformaten zur weiteren Verarbeitung.

**Inhaltsverzeichnis:** 


* [Einrichten der Arbeitsumgebung](#API1)
* [OAI2 - Schnittstelle ](#API2)
* [OAI - Schnittstelle ](#API3)

## Einrichten der Arbeitsumgebung <a class="anchor" id="API1"></a>

Um die Arbeitsumgebung für die folgenden Schritte passend einzurichten, sollten zunächst die benötigten Python-Bibliotheken importiert werden. Für Anfragen über die OAI(2)-Schnittstellen wird sickle https://sickle.readthedocs.io/en/latest/tutorial.html und zur Verarbeitung der XML-Daten etree https://docs.python.org/3/library/xml.etree.elementtree.html verwendet. Mit dem OAIRecordReader https://polymatheia.readthedocs.io/en/latest/api/polymatheia_data_reader.html können Elemente aus dem METS/MODS Format ausgelesen werden.

In [ ]:
from sickle import Sickle
from lxml import etree

## OAI2- Schnittstelle <a class="anchor" id="API2"></a>

Inhaltsverzeichnis:

* [1. Grundlegende Funktionen](#Teil1)
* [2. Ausgabe von Titeln eines Projekts ab bestimmtem Zeitraum](#Teil2)
* [3. Zusätzliche Ausgabe der URNs/Links zu den Objekten](#Teil3)
* [4. Ausgabe der gescannten Bilder](#Teil4)




### 1. Grundlegende Funktionen <a class="anchor" id="Teil1"></a>

Die Daten der frei verfügbaren digitalisierten Objekte werden über die OAI2-Schnittstelle standardmäßig als XML-Antwort in dem Format METS/MODS ausgeliefert. Zusätzlich steht die Auslieferung der Daten in Dublin Core (DC) zur Verfügung.

Die OAI2 Basis URL ist https://services.dnb.de/oai2/repository. 

In [ ]:
sickle = Sickle('https://services.dnb.de/oai2/repository')

Im nächsten Schritt werden alle verfügbaren Objektsammlungen (Projekte) mit der Funktion ListSets abgefragt und angezeigt.

In [ ]:
oai_sets = sickle.ListSets()
for oai_set in oai_sets:
    print('setSpec value for selective harvesting: ' + oai_set.setSpec)
    print('Name of the set (setName): ' + oai_set.setName + '\n')

Die bisher zur Verfügung stehenden freien Objektsammlungen entsprechen folgenden Projekten (oai_sets):
 
| oai_set | Beschreibung | Anzeige im Katalog |
| --- | --- | --- |
| dnb:digitalisate-oa:projekt4 | 100.000 Seiten aus 30 Titeln, die ein breites Spektrum der Exilpresse 1933-1945 repräsentieren | <a href="https://portal.dnb.de/opac.htm?method=showPreviousResultSite&currentResultId=%22exilpresse%22+and+%22digital%22%26any%26dnb.dea.exp%26leipzig&currentPosition=10">Sammlung im Katalog</a>
| dnb:digitalisate-oa:projekt5 | 1.436 Hefte von 26 in NS-Deutschland erschienenen jüdischen Periodika | <a href="https://portal.dnb.de/opac.htm?method=moveDown&currentResultId=cod%3Dd005+and+location%3Donlinefree%26any&categoryId=dnb.dea.jup">Sammlung im Katalog</a>
| dnb:digitalisate-oa:projekt7 | Gemeinfreie Digitalisate der Exilsammlungen | <a href="https://portal.dnb.de/opac.htm?query=cod%3Dd007+location%3Donlinefree+&method=simpleSearch&cqlMode=true">Sammlung im Katalog</a>
| dnb:digitalisate-oa:projekt8 | Mehr als 1.200 Bücher, Broschüren, Plakate und andere Einblattdrucke zum Weltkrieg 1914-1918 | <a href="https://portal.dnb.de/opac.htm?query=cod%3Dd008+location%3Donlinefree+&method=simpleSearch&cqlMode=true">Sammlung im Katalog</a>
| dnb:digitalisate-oa:projekt26 | Mehr als 600 digitalisierte Quellen zur deutschen Revolution 1918/1919 und zur Nachkriegszeit | <a href="https://portal.dnb.de/opac.htm?query=cod%3Dd026+location%3Donlinefree+&method=simpleSearch&cqlMode=true">Sammlung im Katalog</a>
| dnb:digitalisate-oa:projekt28 | Digitalisierte Werke aus Architektur, Design, Malerei und Druckkunst von 1918 bis 1933 | <a href="https://portal.dnb.de/opac.htm?query=cod%3Dd028+location%3Donlinefree+&method=simpleSearch&cqlMode=true">Sammlung im Katalog</a>
| dnb:digitalisate-oa:projekt29 | Mehr als 2.400 Bildnisse von Buchhändlern, Buchdruckern und Verlegern des 17. bis 20. Jahrhunderts | <a href="https://portal.dnb.de/opac.htm?query=cod%3Dd029+location%3Donlinefree+&method=simpleSearch&cqlMode=true">Sammlung im Katalog</a>

Die verfügbaren Metadatenformate können mit der Funktion ListMetadataFormats abgefragt und mit print() ausgegeben werden. 


In [ ]:
oai_formats = sickle.ListMetadataFormats()
for oai_format in oai_formats:
    print(oai_format.metadataPrefix)

### 2. Ausgabe von Titeln eines Projekts in bestimmten Zeitraum <a class="anchor" id="Teil2"></a>

Um beispielhaft alle Datensätze von projekt28 ab dem 01.04.2019 aufzulisten, müssen für die Abfrage Metadatenformat, Zeitraum und Projekt angegeben werden.
Die OAI Funktionen werden unter http://www.openarchives.org/OAI/2.0/ spezifiziert und auf der DNBLab Seite unter Funktionen beschrieben.
Die Daten befinden sich in der XML-Antwort als Inhalt zwischen beschreibenden Elementen und werden durchnummeriert angezeigt. Für die gezielte Abfrage der gewünschten Informationen aus dem Metadatenformat DC können bei Bedarf die DC-Spezifikationen http://www.openarchives.org/OAI/2.0/oai_dc/ und http://purl.org/dc/elements/1.1/ herangezogen werden.


In [ ]:
namespaces = {
    'oai': 'http://www.openarchives.org/OAI/2.0/',
    'oai_dc': 'http://www.openarchives.org/OAI/2.0/oai_dc/',
    'dc': 'http://purl.org/dc/elements/1.1/'
}

count = 0

for record in sickle.ListRecords(**{'metadataPrefix': 'oai_dc','from': '2019-04-30', 'set': 'dnb:digitalisate-oa:projekt28'}):
    if 'ger' in record.raw:
        tree = etree.ElementTree(record.xml)
        result = tree.xpath('/oai:record/oai:metadata/oai_dc:dc/dc:title/text()', namespaces=namespaces)
        if result:
            count += 1
            author = tree.xpath('/oai:record/oai:metadata/oai_dc:dc/dc:creator/text()', namespaces=namespaces)
            print(f"{count}: {result[0]}")
            if author:
                print(author[0])
            

### 3. Zusätzliche Ausgabe der Links zu den Objekten <a class="anchor" id="Teil3"></a>

Damit zusätzlich zu den Titeln die Links zu den Objekten ausgegeben werden, wird der URN entsprechend des dc-Schemas /oai:record/oai:metadata/oai_dc:dc/dc:identifier/text() angesprochen und zusammen mit dem Titel angezeigt. Die Ausgabe der Ergebnisse wird in diesem Beispiel über max_results auf die ersten 10 Treffer limitiert.

In [ ]:
namespaces = {
    'oai': 'http://www.openarchives.org/OAI/2.0/',
    'oai_dc': 'http://www.openarchives.org/OAI/2.0/oai_dc/',
    'dc': 'http://purl.org/dc/elements/1.1/'
}

count = 0
max_results = 10

for record in sickle.ListRecords(**{'metadataPrefix': 'oai_dc', 'from': '2019-04-30', 'set': 'dnb:digitalisate-oa:projekt28'}):
    if count >= max_results:
        break
    
    if ('ger' in record.raw):
        tree = etree.ElementTree(record.xml)
        result = tree.xpath('/oai:record/oai:metadata/oai_dc:dc/dc:title/text()', namespaces=namespaces)
        if (result):
            count += 1
            urn = tree.xpath('/oai:record/oai:metadata/oai_dc:dc/dc:identifier/text()', namespaces=namespaces)
            print(f"{count}: {result[0]}")
            if urn:
                print(f"https://nbn-resolving.org/{urn[0]}")

    if count >= max_results:
        break


### 4. Ausgabe der gescannten Bilder <a class="anchor" id="Teil4"></a>

Das Format METS bildet neben den bibliographischen Daten im Format MODS die Struktur der digitalen Objekte bzw. die Reihenfolge der gescannten Bilder ab. Für die gezielte Abfrage der gewünschten Informationen aus dem Metadatenformat METS können bei Bedarf die METS-Spezifikationen http://www.loc.gov/METS/ und https://www.loc.gov/standards/mets/mets.xsd herangezogen werden. Die Links der einzelnen Bilder (Thumbnails und Defaultbilder) können über <xsd:element name="fileGrp"> angesprochen werden. 

#### 4.1 Ausgabe der Cover-URLs 
In diesem Beispiel wird jeweils das erste gescannte Bild der ersten 10 Treffer (max_records=10) zur weiteren Verarbeitung ausgegeben werden.

In [ ]:
import xml.etree.ElementTree as ET
import requests

# Funktion zum Abrufen der OAI2-Daten
def fetch_oai_records(url, max_records=10, metadata_prefix='mets', set_spec='dnb:digitalisate-oa:projekt28'):
    params = {
        'verb': 'ListRecords',
        'metadataPrefix': metadata_prefix,
        'set': set_spec,
        'resumptionToken': None
    }
    records = []
    
    while len(records) < max_records:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Überprüfen, ob die Anfrage erfolgreich war
        xml_data = response.content
        records.append(xml_data)

        # Überprüfen, ob es einen Resumption Token gibt
        root = ET.fromstring(xml_data)
        resumption_token = root.find('.//{http://www.openarchives.org/OAI/2.0/}resumptionToken')
        
        if resumption_token is not None:
            params['resumptionToken'] = resumption_token.text
        else:
            break

    return records[:max_records]

# Hauptfunktion
def main():
    url = 'http://services.dnb.de/oai2/repository'
    xml_records = fetch_oai_records(url)

    # Namespace für METS definieren
    ns_mets = {'mets': 'http://www.loc.gov/METS/'}
    ns_oai = {'oai': 'http://www.openarchives.org/OAI/2.0/'}

    count = 0
    # Durch die Datensätze iterieren
    for xml_data in xml_records:
        root = ET.fromstring(xml_data)
        
        for record in root.findall('.//oai:record', ns_oai):
            if count >= 10:
                return
            mets_file_sec = record.find('.//mets:fileSec', ns_mets)
            if mets_file_sec is not None:
                mets_file_grp = mets_file_sec.find('.//mets:fileGrp', ns_mets)
                if mets_file_grp is not None:
                    if isinstance(mets_file_grp, list):
                        # Wenn es eine Liste ist, den ersten Eintrag ausgeben
                        mets_file = mets_file_grp[0].find('.//mets:file', ns_mets)
                        if mets_file is not None:
                            flocat = mets_file.find('.//mets:FLocat', ns_mets)
                            if flocat is not None:
                                href = flocat.attrib.get('{http://www.w3.org/1999/xlink}href')
                                if href:
                                    print(href)
                                    count += 1
                    else:
                        # Ansonsten direkt ausgeben
                        mets_file = mets_file_grp.find('.//mets:file', ns_mets)
                        if mets_file is not None:
                            flocat = mets_file.find('.//mets:FLocat', ns_mets)
                            if flocat is not None:
                                href = flocat.attrib.get('{http://www.w3.org/1999/xlink}href')
                                if href:
                                    print(href)
                                    count += 1
            if count >= 10:
                return

if __name__ == "__main__":
    main()


#### 4.2. Ausgabe der Bild-URLS des ersten Dazensatzes
In diesem Beispiel werden alle Bild-URLs des ersten Datensatzes ausgegeben.

In [ ]:
# Funktion zum Abrufen der OAI2-Daten
def fetch_oai_records(url, metadata_prefix='mets', set_spec='dnb:digitalisate-oa:projekt28'):
    params = {
        'verb': 'ListRecords',
        'metadataPrefix': metadata_prefix,
        'set': set_spec
    }
    
    response = requests.get(url, params=params)
    response.raise_for_status()  # Überprüfen, ob die Anfrage erfolgreich war
    return response.content

# Hauptfunktion
def main():
    url = 'http://services.dnb.de/oai2/repository'
    xml_data = fetch_oai_records(url)

    # Namespace für METS und OAI definieren
    ns_mets = {'mets': 'http://www.loc.gov/METS/'}
    ns_oai = {'oai': 'http://www.openarchives.org/OAI/2.0/'}

    root = ET.fromstring(xml_data)
    
    # Den ersten Record finden
    first_record = root.find('.//oai:record', ns_oai)

    #    
    if first_record is not None:
        mets_file_sec = first_record.find('.//mets:fileSec', ns_mets)
        if mets_file_sec is not None:
            mets_file_grp = mets_file_sec.find('.//mets:fileGrp', ns_mets)
            if mets_file_grp is not None:
                for mets_file in mets_file_grp.findall('.//mets:file', ns_mets):
                    flocat = mets_file.find('.//mets:FLocat', ns_mets)
                    if flocat is not None:
                        href = flocat.attrib.get('{http://www.w3.org/1999/xlink}href')
                        if href:
                            print(href)

if __name__ == "__main__":
    main()


Inhaltsverzeichnis:

* [1. Grundlegende Funktionen](#Teil5)
* [2. Ausgabe von Titeln in bestimmten Zeitraum](#Teil6)
* [3. Zusätzliche Ausgabe der Links zu den Objekten](#Teil7)

## OAI- Schnittstelle <a class="anchor" id="API3"></a>

### 1. Grundlegende Funktionen <a class="anchor" id="Teil5"></a>

Die Daten werden über die OAI-Schnittstelle standardmäßig als XML-Antwort in verschiedenen Formaten ausgeliefert.

Die OAI Basis URL ist https://services.dnb.de/oai/repository.

In [ ]:
sickle = Sickle('https://services.dnb.de/oai/repository')

Im nächsten Schritt werden alle verfügbaren Datensets (Kataloge) mit der Funktion ListSets abgefragt und angezeigt.

In [ ]:
oai_sets = sickle.ListSets()
for oai_set in oai_sets:
    print('setSpec value for selective harvesting: ' + oai_set.setSpec)
    print('Name of the set (setName): ' + oai_set.setName + '\n')

Hier eine Auswahl der über OAI verfügbaren Titeldaten (oai_sets):

| Auswahl | Wert für Parameter "set" | 
| --- | --- | 
| Deutsche Nationalbibliografie ohne Gemeinsame Normdatei| dnb:wv (nur Datensätze nach abgeschlossener Bearbeitung) | 
| Deutsche Nationalbibliografie, Reihe A (Publikationen des Verlagsbuchhandels) | dnb:reiheA (inkl. Datensätze in Bearbeitung und abgeschlossene Bearbeitung) |
| Deutsche Nationalbibliografie, Reihe B (Publikationen außerhalb des Verlagsbuchhandels) | dnb:reiheB (inkl. Datensätze in Bearbeitung und abgeschlossene Bearbeitung)
dnb:wv:reiheB (nur Datensätze nach abgeschlossener Bearbeitung) |
| Deutsche Nationalbibliografie, Reihe H (Hochschulschriften)| dnb:reiheH (inkl. Datensätze in Bearbeitung und abgeschlossene Bearbeitung)
dnb:wv:reiheH (nur Datensätze nach abgeschlossener Bearbeitung) |
| Deutsche Nationalbibliografie, Reihe O (Online-Publikationen) | dnb:reiheO (inkl. Datensätze in Bearbeitung und abgeschlossene Bearbeitung)
dnb:wv:reiheO (nur Datensätze nach abgeschlossener Bearbeitung) |
| Online-Publikationen ohne Einschränkungen | dnb-all:online |
| Deutsche Nationalbibliografie: Digitalisierte Inhaltsverzeichnisse| dnb:toc |

Die Daten können über die OAI-Schnittstelle als XML-Antwort in folgenden Formaten ausgeliefert werden: 
* MARC21-xml http://www.loc.gov/standards/marcxml/schema/MARC21slim.xsd
* DNB Casual (oai_dc)	http://www.openarchives.org/OAI/2.0/oai_dc.xsd
* RDF (RDFxml) http://www.w3.org/2000/07/rdf.xsd

### 2. Ausgabe von Titeln eines Projekts in bestimmten Zeitraum <a class="anchor" id="Teil6"></a>
Um beispielhaft Online-Dissertationen aus der Sachgruppe „Sozialwissenschaften, Soziologie, Anthropologie“ ab dem 30.04.2023 aufzulisten, müssen für die Abfrage Metadatenformat, Zeitraum und Projekt angegeben werden. Die OAI Funktionen werden unter http://www.openarchives.org/OAI/2.0/ spezifiziert und auf der DNBLab Seite unter Funktionen beschrieben. Die Daten befinden sich in der XML-Antwort als Inhalt zwischen beschreibenden Elementen und werden durchnummeriert angezeigt. Für die gezielte Abfrage der gewünschten Informationen aus dem Metadatenformat DC, können bei Bedarf die DC-Spezifikationen http://www.openarchives.org/OAI/2.0/oai_dc/ und http://purl.org/dc/elements/1.1/ herangezogen werden.

In [ ]:
namespaces = {
    'oai': 'http://www.openarchives.org/OAI/2.0/',
    'oai_dc': 'http://www.openarchives.org/OAI/2.0/oai_dc/',
    'dc': 'http://purl.org/dc/elements/1.1/'
}

count=0

for record in sickle.ListRecords(**{'metadataPrefix': 'oai_dc', 'from': '2023-04-30', 'set': 'dnb-all:online:dissertations:sg300'}):
    
    if ('ger' in record.raw):
        tree = etree.ElementTree(record.xml)
        result = tree.xpath('/oai:record/oai:metadata/oai_dc:dc/dc:title/text()', namespaces=namespaces)
        if (result):
            count += 1
            author = tree.xpath('/oai:record/oai:metadata/oai_dc:dc/dc:creator/text()', namespaces=namespaces)
            print(str(count) + ": " + result[0])
            if author:
                print(author[0])

### 3. Zusätzliche Ausgabe der Links zu den Objekten <a class="anchor" id="Teil7"></a>
Damit zusätzlich zu den Titeln die Links zu den Objekten ausgegeben werden, wird der URN entsprechend des dc-Schemas /oai:record/oai:metadata/oai_dc:dc/dc:identifier/text() angesprochen und zusammen mit dem Titel angezeigt.

In [ ]:
namespaces = {
    'oai': 'http://www.openarchives.org/OAI/2.0/',
    'oai_dc': 'http://www.openarchives.org/OAI/2.0/oai_dc/',
    'dc': 'http://purl.org/dc/elements/1.1/'
}

count=0

for record in sickle.ListRecords(**{'metadataPrefix': 'oai_dc', 'from': '2024-04-30', 'set': 'dnb-all:online:dissertations:sg300'}):
    
    if ('ger' in record.raw):
        tree = etree.ElementTree(record.xml)
        result = tree.xpath('/oai:record/oai:metadata/oai_dc:dc/dc:title/text()', namespaces=namespaces)
        if (result):
            count += 1
            urn = tree.xpath('/oai:record/oai:metadata/oai_dc:dc/dc:identifier/text()', namespaces=namespaces)
            print(str(count) + ": " + result[0])
            if urn:
                print("https://nbn-resolving.org/" + urn[0])